## 題目

2. Use LSTM & CNN model to classify customized candlestick pattern (at least 3 classes)

### 執行

所有檔案: candlestick_train_cnn.py、candlestick_train_lstm.py

#### 2. Use LSTM model to classify customized candlestick pattern
* candlestick_train_lstm.py

In [ ]:
from sklearn.metrics import confusion_matrix
import pickle
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model

def train_lstm(model, x_train, y_train, x_test, y_test, 
        learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam,
        loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
        batch_size=batch_size, epochs=training_iters,
        verbose=1, validation_data=(x_test, y_test))

def print_result(data, x_train, x_test, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

def mnist_lstm_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 10
    batch_size = 128

    # model parameters
    n_input = 40
    n_step = 10
    n_hidden = 256
    n_classes = 10
#/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl
    data = load_pkl('/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl')
    x_train, y_train, x_test, y_test = data['train_gaf'], data['train_label'][:, 0], data['test_gaf'], data['test_label'][:, 0]
    x_train, x_test, y_train, y_test = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    train_lstm(model, x_train, y_train, x_test, y_test, learning_rate, 
               training_iters, batch_size)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print_result(data, x_train, x_test, model)

In [ ]:
mnist_lstm_main()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               304128    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 306,698
Trainable params: 306,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
118/118 [==============================] - 11s 80ms/step - loss: 2.2088 - accuracy: 0.1919 - val_loss: 1.6876 - val_accuracy: 0.2628
Epoch 2/10
118/118 [==============================] - 9s 76ms/step - loss: 1.5854 - accuracy: 0.3153 - val_loss: 1.3625 - val_accuracy: 0.4158
Epoch 3/10
118/118 [==============================] - 9s 76ms/step - loss: 1.3546 - accurac

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[1615  183  153  158   92  385  212  135   67]
 [  72 1335    0   93    0    0    0    0    0]
 [ 169    0 1314    0   17    0    0    0    0]
 [  30   26    0 1139    0    0    0  305    0]
 [  76    0   61    0 1286    0    0    0   77]
 [ 109    3    0    2    0 1322    2   62    0]
 [ 155    1    1    0    6    0 1269    0   68]
 [  19    2    0  111    0   22    0 1346    0]
 [  57    0    4    0  234    0   42    0 1163]] 

 [[555  42  63  52  31 132  56  46  23]
 [ 22 447   0  31   0   0   0   0   0]
 [ 46   0 450   0   4   0   0   0   0]
 [ 18   4   0 390   0   0   0  88   0]
 [ 24   0  28   0 428   0   0   0  20]
 [ 35   0   0   0   0 457   0   8   0]
 [ 44   0   1   0   1   0 441   0  13]
 [  5   1   0   5   0   8   0 481   0]
 [ 26   0   0   0  55   0   9   0 410]]


#### 3. Use CNN model to classify customized candlestick pattern
* candlestick_train_cnn.py

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle

from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPool2D


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def get_cnn_model(params):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(10, 10, 4)))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    return model

def train_model(params, data):
    model = get_cnn_model(params)
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],
                     batch_size=params['batch_size'], epochs=params['epochs'], verbose=2)
    return (model, hist)

def print_result(data, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(data['train_gaf'])
    test_pred = model.predict_classes(data['test_gaf'])
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

In [ ]:
PARAMS = {}
PARAMS['pkl_name'] = '/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl'
PARAMS['classes'] = 9
PARAMS['lr'] = 0.01
PARAMS['epochs'] = 10
PARAMS['batch_size'] = 64
PARAMS['optimizer'] = optimizers.SGD(lr=PARAMS['lr'])

# ---------------------------------------------------------
# load data & keras model
data = load_pkl(PARAMS['pkl_name'])
# train cnn model
model, hist = train_model(PARAMS, data)
# train & test result
scores = model.evaluate(data['test_gaf'], data['test_label_arr'], verbose=0)
print('CNN test accuracy:', scores[1])
print_result(data, model)

Epoch 1/10
235/235 - 7s - loss: 1.7124 - accuracy: 0.3529
Epoch 2/10
235/235 - 7s - loss: 0.8493 - accuracy: 0.6951
Epoch 3/10
235/235 - 6s - loss: 0.6263 - accuracy: 0.7745
Epoch 4/10
235/235 - 6s - loss: 0.5438 - accuracy: 0.8023
Epoch 5/10
235/235 - 6s - loss: 0.5044 - accuracy: 0.8171
Epoch 6/10
235/235 - 7s - loss: 0.4736 - accuracy: 0.8275
Epoch 7/10
235/235 - 7s - loss: 0.4546 - accuracy: 0.8340
Epoch 8/10
235/235 - 7s - loss: 0.4352 - accuracy: 0.8409
Epoch 9/10
235/235 - 7s - loss: 0.4196 - accuracy: 0.8471
Epoch 10/10
235/235 - 7s - loss: 0.4108 - accuracy: 0.8503
CNN test accuracy: 0.8575999736785889


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[2257  124  115   91   97  102   73   96   45]
 [  25 1474    0    1    0    0    0    0    0]
 [  74    0 1418    0    8    0    0    0    0]
 [  64   61    0 1292    0    0    0   83    0]
 [  66    0   29    0 1342    0    5    0   58]
 [  79    1    0    0    0 1383    1   36    0]
 [ 172    2    2    0    2    0 1298    0   24]
 [  44   10    0  139    0    8    0 1299    0]
 [ 107    0    2    0  190    0   46    0 1155]] 

 [[753  36  45  28  33  34  15  39  17]
 [  9 491   0   0   0   0   0   0   0]
 [ 22   0 478   0   0   0   0   0   0]
 [ 26  20   0 426   0   0   0  28   0]
 [ 35   0  10   0 424   0   1   0  30]
 [ 32   1   0   0   0 460   0   7   0]
 [ 74   0   1   0   0   0 423   0   2]
 [ 20   1   0  29   0   5   0 445   0]
 [ 52   0   0   0  45   0  15   0 388]]


## Reference